**Реализация Identification Rate Metric**

В данном ноутбуке реализована функция, считающая Identification Rate Metric. Проверена правильность её работы на тестовых данных. Посчитаны значения метрики для двух обученных моделей:


*   resnet34, обученной на CE loss
*   resnet34, обученной на ArcFace loss



# Загрузка данных и импорт необходимых библиотек

Для подсчёта Identification Rate Metric используем следующие данные из датасета CelebA-500: https://disk.yandex.com/d/KN4EEkNKrF_ZXQ
Эти данные уже разбиты на query и distractors, и в отдельном файле также находится информация о классах для картинок из query. Эти картинки заалайнены точно так же, как картинки из обучающей выборки CelebA-500

In [1]:
import torch
from torchvision.models import resnet34
import torchvision.transforms as tt
from PIL import Image
import tqdm
import numpy as np
import torch.nn as nn

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip -uq "/content/drive/MyDrive/Datasets Deep Learning/celebA_ir.zip" -d "/content/celebA_ir"

С помощью следующей ячейки загрузим данные

In [5]:
from collections import defaultdict
import os

# file with query part annotations: which image belongs to which class
# format:
#     image_name_1.jpg 2678
#     image_name_2.jpg 2679
f = open('/content/celebA_ir/celebA_ir/celebA_anno_query.csv', 'r')
query_lines = f.readlines()[1:]
f.close()
query_lines = [x.strip().split(',') for x in query_lines]
# plain list of image names from query. Neede to compute embeddings for query
query_img_names = [x[0] for x in query_lines]

# dictionary with info of which images from query belong to which class
# format:
#     {class: [image_1, image_2, ...]}
query_dict = defaultdict(list)
for img_name, img_class in query_lines:
  query_dict[img_class].append(img_name)

# list of distractor images
distractors_img_names = os.listdir('/content/celebA_ir/celebA_ir/celebA_distractors')

In [6]:
len(distractors_img_names)

2001

In [7]:
len(query_img_names)

1222

# Реализация IR metric и вспомогательных функций для её подсчёта

**Описание того, как работает метрика IR**

Создадим два набора изображений лиц: query и distractors. Никакие лица из этих наборов не должны содержаться в обучающем и валидационном датасете.

1. посчитаем косинусные расстояния между лицами, соответствующими одним и тем же людям из query части. Например, пусть одному человеку соответствуют три фото в query: 01.jpg, 02.jpg, 03.jpg. Тогда считаем три косинусных расстояния между всеми тремя парами из этих фото.
2. посчитаем косинусные расстояния между лицами, соответствующими разным людям из query части.
3. посчитаем косинусные расстояния между всеми парами лиц из query и distractors. Т.е. пара — это (лицо из query, лицо из distractors). Всего получится |query|*|distractors| пар.
4. Сложим количества пар, полученных на 2 и 3 шагах. Это количество false пар.
5. Зафиксируем **FPR** (false positive rate). Пусть, например, будет 0.01. FPR, умноженный на количество false пар из шага 4 — это разрешенное количество false positives, которые мы разрешаем нашей модели. Обозначим это количество через N.
6. Отсортируем все значения косинусных расстояний false пар. N — ое по счету значение расстояния зафиксируем как **пороговое расстояние**.
7. Посчитаем количество positive пар с шага 1, которые имеют косинусное расстояние меньше, чем пороговое расстояние. Поделим это количество на общее количество positive пар с шага 1. Это будет TPR (true positive rate) — итоговое значение нашей метрики.

In [8]:
def compute_embeddings(model, images_list, image_folder):
  '''
  compute embeddings from the trained model for list of images.
  params:
    model: trained nn model that takes images and outputs embeddings
    images_list: list of images paths to compute embeddings for
  output:
    image_folder: path to image folder of images
    list: list of model embeddings. Each embedding corresponds to images
          names from images_list
  '''
  model.eval()
  preprocess = tt.Compose([
        tt.Resize((224, 224)),
        tt.ToTensor(),
        tt.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

  embeddings_list = []
  for image_name in tqdm.tqdm(images_list):
    image_path = f'{image_folder}/{image_name}'
    image = Image.open(image_path)

    input_tensor = preprocess(image).unsqueeze(0)

    model = model.to(device)
    input_tensor = input_tensor.to(device)

    with torch.no_grad():
      embedding = model(input_tensor)
      embeddings_list.append(embedding.cpu().numpy().flatten())

  return embeddings_list

In [9]:
def compute_cosine_similarity(embedding1, embedding2):
  dot_result = np.dot(embedding1, embedding2)
  norm1 = np.linalg.norm(embedding1)
  norm2 = np.linalg.norm(embedding2)
  return dot_result / (norm1 * norm2)

Реализуем **пункт 1**: посчитаем косинусные расстояния между лицами, соответствующими одним и тем же людям из query части

In [10]:
def compute_cosine_query_pos(query_dict, query_img_names, query_embeddings):
  '''
  compute cosine similarities between positive pairs from query (stage 1)
  params:
    query_dict: dict {class: [image_name_1, image_name_2, ...]}. Key: class in
                the dataset. Value: images corresponding to that class
    query_img_names: list of images names
    query_embeddings: list of embeddings corresponding to query_img_names
  output:
    list of floats: similarities between embeddings corresponding
                    to the same people from query list
  '''
  positive_similarities = []

  # словарь где ключ -- имя изображения, значение -- соответствующий эмбеддинг
  img_to_embedding = {name: embedding for name, embedding in zip(query_img_names, query_embeddings)}

  for class_name, image_names in tqdm.tqdm(query_dict.items()):
    for i in range(len(image_names)):
      for j in range(i + 1, len(image_names)):
        emb1 = img_to_embedding[image_names[i]]
        emb2 = img_to_embedding[image_names[j]]
        similarity = compute_cosine_similarity(emb1, emb2)
        positive_similarities.append(similarity)

  return positive_similarities

Реализуем **пункт 2**: посчитаем косинусные расстояния между лицами, соответствующими разным людям из query части.

In [11]:
def compute_cosine_query_neg(query_dict, query_img_names, query_embeddings):
  '''
  compute cosine similarities between negative pairs from query (stage 2)
  params:
    query_dict: dict {class: [image_name_1, image_name_2, ...]}. Key: class in
                the dataset. Value: images corresponding to that class
    query_img_names: list of images names
    query_embeddings: list of embeddings corresponding to query_img_names
  output:
    list of floats: similarities between embeddings corresponding
                    to different people from query list
  '''
  negative_similarities = []

  # словарь где ключ -- имя изображения, значение -- соответствующий эмбеддинг
  img_to_embedding = {name: embedding for name, embedding in zip(query_img_names, query_embeddings)}

  # будем хранить в списке названия всех классов для удобства итерации
  img_classes_list = list(query_dict.keys())

  for i in tqdm.tqdm(range(len(img_classes_list))):
    class1_name = img_classes_list[i]
    class1_images = query_dict[class1_name]
    for j in range(i+1, len(img_classes_list)):
      class2_name = img_classes_list[j]
      class2_images = query_dict[class2_name]
      for image1_name in class1_images:
        emb1 = img_to_embedding[image1_name]
        for image2_name in class2_images:
          emb2 = img_to_embedding[image2_name]
          similarity = compute_cosine_similarity(emb1, emb2)
          negative_similarities.append(similarity)

  return negative_similarities


**Пункт 3**: посчитаем косинусные расстояния между всеми парами лиц из query и distractors. Всего получится |query|*|distractors| пар.

In [12]:
def compute_cosine_query_distractors(query_embeddings, distractors_embeddings):
  '''
  compute cosine similarities between negative pairs from query and distractors
  (stage 3)
  params:
    query_embeddings: list of embeddings corresponding to query_img_names
    distractors_embeddings: list of embeddings corresponding to distractors_img_names
  output:
    list of floats: similarities between pairs of people (q, d), where q is
                    embedding corresponding to photo from query, d —
                    embedding corresponding to photo from distractors
  '''
  cosine_similarities = []

  for query_embedding in tqdm.tqdm(query_embeddings):
    for distractors_embedding in distractors_embeddings:
      similarity = compute_cosine_similarity(query_embedding, distractors_embedding)
      cosine_similarities.append(similarity)
  return cosine_similarities

Проверка того, что код работает верно

In [13]:
test_query_dict = {
    2876: ['1.jpg', '2.jpg', '3.jpg'],
    5674: ['5.jpg'],
    864:  ['9.jpg', '10.jpg'],
}
test_query_img_names = ['1.jpg', '2.jpg', '3.jpg', '5.jpg', '9.jpg', '10.jpg']
test_query_embeddings = [
                    [1.56, 6.45,  -7.68],
                    [-1.1 , 6.11,  -3.0],
                    [-0.06,-0.98,-1.29],
                    [8.56, 1.45,  1.11],
                    [0.7,  1.1,   -7.56],
                    [0.05, 0.9,   -2.56],
]

test_distractors_img_names = ['11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg']

test_distractors_embeddings = [
                    [0.12, -3.23, -5.55],
                    [-1,   -0.01, 1.22],
                    [0.06, -0.23, 1.34],
                    [-6.6, 1.45,  -1.45],
                    [0.89,  1.98, 1.45],
]

test_cosine_query_pos = compute_cosine_query_pos(test_query_dict, test_query_img_names,
                                            test_query_embeddings)
test_cosine_query_neg = compute_cosine_query_neg(test_query_dict, test_query_img_names,
                                            test_query_embeddings)
test_cosine_query_distractors = compute_cosine_query_distractors(test_query_embeddings,
                                                            test_distractors_embeddings)

100%|██████████| 6/6 [00:00<00:00, 8879.97it/s]


In [14]:
true_cosine_query_pos = [0.8678237233650096, 0.21226104378511604,
                         -0.18355866977496182, 0.9787437979250561]
assert np.allclose(sorted(test_cosine_query_pos), sorted(true_cosine_query_pos)), \
      "A mistake in compute_cosine_query_pos function"

true_cosine_query_neg = [0.15963231223161822, 0.8507997093616965, 0.9272761484302097,
                         -0.0643994061127092, 0.5412660901220571, 0.701307100338029,
                         -0.2372575528216902, 0.6941032794522218, 0.549425446066643,
                         -0.011982733001947084, -0.0466679194884999]
assert np.allclose(sorted(test_cosine_query_neg), sorted(true_cosine_query_neg)), \
      "A mistake in compute_cosine_query_neg function"

true_cosine_query_distractors = [0.3371426578637511, -0.6866465610863652, -0.8456563512871669,
                                 0.14530087113136106, 0.11410510307646118, -0.07265097629002357,
                                 -0.24097699660707042,-0.5851992679925766, 0.4295494455718534,
                                 0.37604478596058194, 0.9909483738948858, -0.5881093317868022,
                                 -0.6829712976642919, 0.07546364489032083, -0.9130970963915521,
                                 -0.17463101988684684, -0.5229363015558941, 0.1399896725311533,
                                 -0.9258034013399499, 0.5295114163723346, 0.7811585442749943,
                                 -0.8208760031249596, -0.9905139680301821, 0.14969764653247228,
                                 -0.40749654525418444, 0.648660814944824, -0.7432584300096284,
                                 -0.9839696492435877, 0.2498741082804709, -0.2661183373780491]
assert np.allclose(sorted(test_cosine_query_distractors), sorted(true_cosine_query_distractors)), \
      "A mistake in compute_cosine_query_distractors function"

In [15]:
def compute_ir(cosine_query_pos, cosine_query_neg, cosine_query_distractors,
               fpr=0.1):
  '''
  compute identification rate using precomputer cosine similarities between pairs
  at given fpr
  params:
    cosine_query_pos: cosine similarities between positive pairs from query
    cosine_query_neg: cosine similarities between negative pairs from query
    cosine_query_distractors: cosine similarities between negative pairs
                              from query and distractors
    fpr: false positive rate at which to compute TPR
  output:
    float: threshold for given fpr
    float: TPR at given FPR
  '''
  false_pairs_number = len(cosine_query_neg) + len(cosine_query_distractors) # количество false negative pairs
  N = fpr * false_pairs_number # разрешённое количество false positives
  N = int(round(N))
  false_pairs = cosine_query_neg + cosine_query_distractors
  false_pairs.sort(reverse = True) # отсортируем все значения косинусных расстояний false пар

  # N — ое по счету значение расстояния зафиксируем как пороговое расстояние.
  if N >= len(false_pairs):
    threshold = false_pairs[-1]
  else:
    threshold = false_pairs[N]
  # теперь посчитаем метрику tpr согласно пункту 7 описания метрики
  TPR = len([True for i in cosine_query_pos if i >= threshold]) / len(cosine_query_pos)
  return threshold, TPR

С помощью ячеек ниже проверим, что метрика посчитана правильно

In [16]:
test_thr = []
test_tpr = []
for fpr in [0.5, 0.3, 0.1]:
  x, y = compute_ir(test_cosine_query_pos, test_cosine_query_neg,
                    test_cosine_query_distractors, fpr=fpr)
  test_thr.append(x)
  test_tpr.append(y)

In [17]:
true_thr = [-0.011982733001947084, 0.3371426578637511, 0.701307100338029]
assert np.allclose(np.array(test_thr), np.array(true_thr)), "A mistake in computing threshold"

true_tpr = [0.75, 0.5, 0.5]
assert np.allclose(np.array(test_tpr), np.array(true_tpr)), "A mistake in computing tpr"

In [18]:
compute_ir(test_cosine_query_pos, test_cosine_query_neg, test_cosine_query_distractors,
               fpr=0.3)

(0.33714265786375097, 0.5)

Все проверки пройдены

# Импорт обученной на CE модели resnet34, сохранённой на 50 - й эпохе обучения

In [27]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [29]:
model = resnet34()

Мы уже обучили модель на CE Loss. Теперь загрузим состояние этой модели на последней, 50 - й эпохе. Загрузка производится на cpu.

In [30]:
load_model_state = torch.load('/content/model_state_dict_epoch_50.pt', map_location=torch.device('cpu'))

<ipython-input-30-dbafb3f7ae6c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_model_state = torch.load('/content/model_state_dict_epoch_50.pt', map_location=torch.de

Заменяем fully connected слой на Dropout + линейный слой с нужным нам количеством классов - 500.

In [31]:
in_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(in_features, 500)
)
model.load_state_dict(load_model_state['state_model']) # загружаем сохранённое состояние модели

<All keys matched successfully>

In [32]:
backbone = torch.nn.Sequential(*list(model.children())[:-1]) # создаём итератор по всем слоям модели с помощью model.children() и оставляем все слои кроме последнего классификационного

In [33]:
image_folder_query = '/content/celebA_ir/celebA_ir/celebA_query'
image_folder_distractors = '/content/celebA_ir/celebA_ir/celebA_distractors'

Найдём эмбеддинги для наших данных

In [34]:
query_embeddings = compute_embeddings(backbone, query_img_names, image_folder_query)
distractors_embeddings = compute_embeddings(backbone, distractors_img_names, image_folder_distractors)

100%|██████████| 2001/2001 [07:14<00:00,  4.61it/s]


Посчитаем cosine_query_pos, cosine_query_neg, cosine_query_distractors для наших данных

In [35]:
cosine_query_pos = compute_cosine_query_pos(query_dict, query_img_names,
                                            query_embeddings)

100%|██████████| 51/51 [00:00<00:00, 424.71it/s]


In [36]:
cosine_query_neg = compute_cosine_query_neg(query_dict, query_img_names,
                                            query_embeddings)

100%|██████████| 51/51 [00:06<00:00,  8.07it/s]


In [37]:
cosine_query_distractors = compute_cosine_query_distractors(query_embeddings,
                                                            distractors_embeddings)

100%|██████████| 1222/1222 [00:24<00:00, 50.12it/s]


Посчитаем TPR@FPR для датасета с лицами для значений fpr = [0.5, 0.2, 0.1, 0.05].

In [38]:
for FPR in [0.5, 0.2, 0.1, 0.05]:
  thr, tpr = compute_ir(cosine_query_pos, cosine_query_neg, cosine_query_distractors, fpr=FPR)
  print(f'IR metric = {tpr} при frp = {FPR}')

IR metric = 0.924325793084709 при frp = 0.5
IR metric = 0.7471639154948713 при frp = 0.2
IR metric = 0.6060050268324163 при frp = 0.1
IR metric = 0.48373072481489027 при frp = 0.05


# Импорт обученной на ArcLoss модели resnet34, сохранённой на 40 - й эпохе обучения

In [41]:
import torch.nn.functional as F
import math
import torchvision.models as models
import torch.nn as nn

In [42]:
class ArcFace(nn.Module):
    """
    Имплементация ArcFace Loss (Additive Angular Margin Loss)

    Параметры:
        in_features (int): размерность входных эмбеддингов (512 в случае resnet34)
        out_features (int): количество классов
        s (float): scale коэффициент для логитов (default 64.0)
        m (float): margin, добавляемый для улучшения разделимости классов (default 0.5)
    """

    def __init__(self, in_features, out_features, s=64.0, m=0.5):
        super(ArcFace, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, label):
        '''
        Параметры:
            input (torch.Tensor): эмбеддинги размерности [batch_size, in_features].
            label (torch.Tensor): истинные метки классов размерности [batch_size].

        Возвращает:
            torch.Tensor: модифицированные логиты, которые мы уже будем подавать в softmax
        '''

        input = F.normalize(input, p = 2, dim = 1)
        weight = F.normalize(self.weight, p = 2, dim = 1)
        cosine = F.linear(input, weight)
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        alpha = cosine * math.cos(self.m) - sine * math.sin(self.m)
        one_hot = torch.zeros(cosine.size(), device=device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        output = (one_hot * alpha) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [43]:
class ArcFaceModel(nn.Module):
    '''
    Модель ArcFace, созданная на основе resnet34
    '''
    def __init__(self):
        super(ArcFaceModel, self).__init__()

        self.backbone = models.resnet34(weights = 'DEFAULT')
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, 512)
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.arcface = ArcFace(512, 500)

    def forward(self, x, labels=None):
        '''
        Параметры:
            x (torch.Tensor): входные изображения размерности [batch_size, 3, H, W]
            labels (torch.Tensor, optional): истинные метки классов. При их передаче используется слой ArcFace
        Возвращает:
            torch.Tensor: возвращает эмбеддинги (при отсутствии labels), логиты (при наличии labels)
        '''

        x = self.backbone(x)
        x = self.batch_norm1(x)
        if labels is not None:
            x = self.arcface(x, labels)
        return x

In [45]:
load_model_state = torch.load('model_state_dict_epoch_40.pt', map_location=torch.device('cpu'))

<ipython-input-45-8a387a41f2c8>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_model_state = torch.load('model_state_dict_epoch_40.pt', map_location=torch.device('cpu

In [46]:
arcface_model = ArcFaceModel()

In [47]:
arcface_model.load_state_dict(load_model_state['state_model'])

<All keys matched successfully>

In [48]:
query_embeddings = compute_embeddings(arcface_model, query_img_names, image_folder_query)
distractors_embeddings = compute_embeddings(arcface_model, distractors_img_names, image_folder_distractors)

100%|██████████| 2001/2001 [06:46<00:00,  4.92it/s]


In [49]:
cosine_query_pos = compute_cosine_query_pos(query_dict, query_img_names,
                                            query_embeddings)

100%|██████████| 51/51 [00:00<00:00, 228.76it/s]


In [50]:
cosine_query_neg = compute_cosine_query_neg(query_dict, query_img_names,
                                            query_embeddings)

100%|██████████| 51/51 [00:07<00:00,  6.85it/s]


In [51]:
cosine_query_distractors = compute_cosine_query_distractors(query_embeddings,
                                                            distractors_embeddings)

100%|██████████| 1222/1222 [00:21<00:00, 57.35it/s]


In [52]:
for FPR in [0.5, 0.2, 0.1, 0.05]:
  thr, tpr = compute_ir(cosine_query_pos, cosine_query_neg, cosine_query_distractors, fpr=FPR)
  print(f'IR metric = {tpr} при frp = {FPR}')

IR metric = 0.8578221588207323 при frp = 0.5
IR metric = 0.6296447252224713 при frp = 0.2
IR metric = 0.4740167108212757 при frp = 0.1
IR metric = 0.3558861490387881 при frp = 0.05
